In [ ]:
import pandas as pd
import numpy as np
import re
import os
import time


raw_path = 'C:\\Finance_team\\raw_data'
out_path = 'C:\\Finance_team\\Testing_result'
file_list = os.listdir('C:\\Finance_team\\raw_data')

result_file = open(out_path + '\\profit_loss_KR4165N30007_김종재.csv', 'a', encoding='ISO-8859-1')
result_file.write('Trading_Date,Daily_P&L,Order_Counts,Short_Counts,Short_Close_Counts,Long_Counts,Long_Close_Counts,Long_Close_Count_at_34,Short_Close_Count_at_34\n')
result_file.close()

product_id = ''
product_code = ''
date = ''

Accumulated_Profitandloss = ''

all_columns = ['시간', '현재가', '체결량', '투자자', '매수1호가수량', '매도1호가수량',
               '투자자매수량', '투자자매도량', '매도or매수']
int_columns = ['현재가', '체결량', '투자자정보', '매수1호가수량', '매도1호가수량',
               '투자자매수량', '투자자매도량', '매도or매수']

def is_target_product_ID(line):
    return line[17:19] == 'A0' and line[425:428] == 'BM3' and line[471:474] == '001'


for i in file_list:
    print(i)
    with open('{0}\\{1}'.format(raw_path, i), 'r', encoding='ISO-8859-1') as f:
        result = []
        lines = f.readlines()
        data_in_array = []

        for line in lines:
            if is_target_product_ID(line):
                product_id = line[53:64]
                product_code = line[35:47]
                date = line[27:35]
                break


        for line in lines:
            data_in_array.append(np.array(line.split(',')))

        for line in data_in_array:

            if (line[0][1:3] < '09'): continue

            if (line[0][1:3] == '15' and line[0][4:6] == '45' and line[0][7:9] >= '01'): break

            if (line[0][22:34] == product_code) or (line[0][38:49] == product_id):

                if line[0][17:19] == 'A3':
                    result.append([line[0][:17], line[0][40:48], line[0][48:54],np.nan,
                                    np.nan, np.nan, np.nan, np.nan, line[0][152:153]])
                elif line[0][17:19] == 'B6':
                    result.append([line[0][:17], np.nan, np.nan, np.nan, line[0][57:63],
                                    line[0][139:145], np.nan, np.nan, np.nan])
                elif line[0][17:19] == 'G7':
                    result.append([line[0][:17], line[0][40:48], line[0][48:54],
                                   np.nan, line[0][169:175], line[0][251:257],
                                   np.nan, np.nan, line[0][152:153]])
                elif (line[0][17:19] == 'H1' and (line[0][49:53] == '1000')):
                    if (line[0][1:3] == '15' and line[0][4:6] == '35' and line[0][7:9] >= '01'): break
                    result.append([line[0][:17], np.nan, np.nan, line[0][49:53],
                                  np.nan, np.nan, line[0][53:62], line[0][62:71],
                                  np.nan])

        result_frame = pd.DataFrame(result, columns = all_columns)
        result_frame.drop(0, inplace=True)


        h1 = result_frame.copy()
        h1 = h1[(h1['투자자'] == '1000')]
        h1.iloc[:, 1:] = h1.iloc[:, 1:].astype('float')

        h1 = h1[['시간', '투자자매수량', '투자자매도량']]
        # h1['투자자매수량변화(%)'] = round(h1['투자자매수량'].pct_change() * 100, 1)
        # h1['투자자매도량변화(%)'] = round(h1['투자자매도량'].pct_change() * 100, 2)
        h1['투자자매수량변화'] = h1['투자자매수량'].diff()
        h1['투자자매도량변화'] = h1['투자자매도량'].diff()
        h1['투자자변화량차이'] = h1['투자자매수량변화'] - h1['투자자매도량변화']

        h1['H1_Flag'] = np.nan
        h1['H1_Flag'][h1['투자자변화량차이'] > 0] = 1
        h1['H1_Flag'][h1['투자자변화량차이'] < 0] = -1
        h1['H1_Flag'][h1['투자자변화량차이'] == 0] = 0

        # 먼저 Current Price정리
        rest = result_frame.copy()
        rest = rest[rest['투자자'] != '1000']
        rest = rest[['시간', '현재가', '체결량', '매수1호가수량', '매도1호가수량', '매도or매수']]
        rest.iloc[:, 1:] = rest.iloc[:, 1:].astype('float')

        # rest['현재가'].fillna(method='ffill', inplace=True)
        rest['매수1호가수량-매도1호가수량'] = rest['매수1호가수량'] - rest['매도1호가수량']
        rest['Rest_Flag'] = np.nan
        rest["Rest_Flag"][rest['매수1호가수량-매도1호가수량'] > 0] = 1
        rest["Rest_Flag"][rest['매수1호가수량-매도1호가수량'] == 0] = 0
        rest["Rest_Flag"][rest['매수1호가수량-매도1호가수량'] < 0] = -1


        ##
        df2 = pd.concat([rest, h1])
        df2.sort_index(inplace=True)

        cols = ['시간', '현재가', '체결량', '매도or매수', '매수1호가수량', '매도1호가수량',
               '매수1호가수량-매도1호가수량', 'Rest_Flag', '투자자매수량', '투자자매도량',
               '투자자매수량변화', '투자자매도량변화', '투자자변화량차이', 'H1_Flag']

        df_test = df2[cols]

        df_test['현재가'].fillna(method='ffill', inplace=True)
        df_test['H1_Flag'].fillna(method='ffill', inplace=True)
        df_test['Rest_Flag'].fillna(method='ffill', inplace=True)

        a = df_test[['시간', '현재가', '매수1호가수량', '매도1호가수량', 'H1_Flag', 'Rest_Flag']]

        a['H1_Flag'][a['H1_Flag'] == 0] = np.nan
        a['Rest_Flag'][a['Rest_Flag'] == 0] = np.nan

        a['H1_Flag'].fillna(method='ffill', inplace=True)
        a['Rest_Flag'].fillna(method='ffill', inplace=True)

        a.loc[:,'long_signal'] = np.nan
        a.loc[:,'long_close_signal'] = np.nan

        renew_index_long = 0
        holding_position = 0
        Daily_profitandloss = 0

        order_count = 0

        long_count = 0
        long_close_count = 0
        long_close_at_34 = 0

        short_count = 0
        short_close_count = 0
        short_close_at_34 = 0


        while True:
            for index, row in a.loc[renew_index_long:].iterrows():


            ######################################## Holding_Position == 0 ################################################

                if holding_position == 0:


                    ################################### 'Buy SIgnal Start' ##################################################


                    if row['H1_Flag'] == -1 and row['Rest_Flag'] == 1 and (row['시간'][1:3] <= '15' and row['시간'][4:6] < '34'):
                        renew_index_long = index
                        # print('Long Signal Found')
                        # print(row['H1_Flag'], row['Rest_Flag'], 'InDex:', index)

                        for index, row in a.loc[renew_index_long:].iterrows():

                            if row['H1_Flag'] == -1 and row['Rest_Flag'] == -1 and (row['시간'][1:3] <= '15' and row['시간'][4:6] < '34'):
                                renew_index_long = index

                                holding_position += 1
                                order_count += 1
                                long_count += 1

                                price_Long = a.loc[index,'현재가']

                                # print(row['H1_Flag'], row['Rest_Flag'], 'InDex:', index, 'Long 가격:', price_Long)
                                # print('Current Holding Position', holding_position)

                                for index, row in a.loc[renew_index_long:].iterrows():

                                    if row['H1_Flag'] == 1 and row['Rest_Flag'] == -1 and (row['시간'][1:3] <= '15' and row['시간'][4:6] < '34'):
                                        renew_index_long = index
                                        # print('Long Closing Signal Found')
                                        # print(row['H1_Flag'], row['Rest_Flag'], 'InDex:', index)

                                        for index, row in a.loc[renew_index_long:].iterrows():

                                            if row['H1_Flag'] == 1 and row['Rest_Flag'] == 1 and (row['시간'][1:3] <= '15' and row['시간'][4:6] < '34'):
                                                renew_index_long = index

                                                holding_position -= 1

                                                order_count += 1
                                                long_close_count += 1

                                                price_Closing_Long = a.loc[index,'현재가']

                                                Long_p_and_l = price_Closing_Long - price_Long

                                                Daily_profitandloss += Long_p_and_l

                                                # print(row['H1_Flag'] , row['Rest_Flag'], 'InDex:', index,
                                                #       'Closing_Long_가격:',price_Closing_Long, 'Long Profit:', Long_p_and_l)
                                                # print('Current Holding Position', holding_position)
                                                # print('Long Closing Signal Finding is All Done')
                                                # print('########################################')
                                                break
                                        break

                                    elif (a.loc[renew_index_long, '시간'][1:3] == '15') and (a.loc[renew_index_long, '시간'][4:6] == '34') and holding_position != 0:
                                        holding_position -= 1
                                        order_count += 1
                                        long_close_count += 1
                                        long_close_at_34 += 1
                                        price_Closing_Long_at_34 = a.loc[index, '현재가']
                                        Long_p_and_l = price_Closing_Long_at_34 - price_Long
                                        Daily_profitandloss += Long_p_and_l

                                        # print(row['H1_Flag'], row['Rest_Flag'], 'InDex:', index,
                                        #           'Closing_Long_at_34_가격:', price_Closing_Long_at_34, '34_Long Profit:', Long_p_and_l)
                                        # print('Current Holding Position', holding_position)

                                        break

                                break
                        break

                    ################################### 'Short SIgnal Start' ##################################################

                    elif row['H1_Flag'] == 1 and row['Rest_Flag'] == -1 and (row['시간'][1:3] <= '15' and row['시간'][4:6] < '34'):
                        renew_index_long = index
                        # print('Short Signal Found')
                        # print(row['H1_Flag'], row['Rest_Flag'], 'InDex:', index)

                        for index, row in a.loc[renew_index_long:].iterrows():

                            if row['H1_Flag'] == 1 and row['Rest_Flag'] and (row['시간'][1:3] <= '15' and row['시간'][4:6] < '34') == 1:

                                renew_index_long = index
                                holding_position += 1
                                order_count += 1
                                short_count += 1

                                price_Short = a.loc[index,'현재가']

                                # print(row['H1_Flag'], row['Rest_Flag'], 'InDex:', index, 'Short 가격:', price_Short)
                                # print('Current Holding Position', holding_position)

                                for index, row in a.loc[renew_index_long:].iterrows():

                                    if row['H1_Flag'] == -1 and row['Rest_Flag'] == 1 and (row['시간'][1:3] <= '15' and row['시간'][4:6] < '34'):
                                        renew_index_long = index
                                        # print('Short Closing Signal Found')
                                        # print(row['H1_Flag'], row['Rest_Flag'], 'InDex:', index)

                                        for index, row in a.loc[renew_index_long:].iterrows():

                                            if row['H1_Flag'] == -1 and row['Rest_Flag'] == -1 and (row['시간'][1:3] <= '15' and row['시간'][4:6] < '34'):
                                                renew_index_long = index

                                                holding_position -= 1

                                                order_count += 1
                                                short_close_count += 1

                                                price_Closing_Short = a.loc[index,'현재가']

                                                Short_p_and_l = price_Short - price_Closing_Short

                                                Daily_profitandloss += Short_p_and_l

                                                # print(row['H1_Flag'] , row['Rest_Flag'], 'InDex:', index,
                                                #       'Closing_Short_가격:',price_Closing_Short, 'Short Profit:', Short_p_and_l)
                                                # print('Current Holding Position', holding_position)
                                                # print('Short Closing Signal Finding is All Done')
                                                # print('########################################')

                                                break
                                        break

                                    elif (a.loc[renew_index_long, '시간'][1:3] == '15') and (a.loc[renew_index_long, '시간'][4:6] == '34') and holding_position != 0:
                                            holding_position -= 1
                                            order_count += 1
                                            short_close_count += 1
                                            short_close_at_34 += 1

                                            price_Closing_Short_at_34 = a.loc[index, '현재가']
                                            Short_p_and_l = price_Short - price_Closing_Short_at_34
                                            Daily_profitandloss += Short_p_and_l

                                            # print(row['H1_Flag'], row['Rest_Flag'], 'InDex:', index,
                                            #       'Closing_Short_at_34_가격:', price_Closing_Short_at_34, '34_Short Profit:', Short_p_and_l)
                                            # print('Current Holding Position', holding_position)

                                            break

                                break
                        break


            if index == len(a.loc[:]):
                break
        result_file = open(out_path + '\\profit_loss_KR4165N30007_김종재.csv', 'a', encoding='ISO-8859-1')
        result_file.write('{0},{1},{2},{3},{4},{5},{6},{7},{8}\n'.format(date, Daily_profitandloss, order_count,
                          short_count, short_close_count, long_count, long_close_count,
                         long_close_at_34, short_close_at_34))
        result_file.close()
